### 使用Deepwave进行Waveform Modeling

在本例中，我们将加载Marmousi模型，指定震源和接收器位置，然后使用Deepwave执行正向建模，通过模型传播源，生成合成接收器数据。

因为我没有搞定Deepwave软件在Windows系统下的编译问题，故注释掉。
测试成功的系统：Ubuntu20.04，Centos7和Centos8

In [ ]:
import torch
import matplotlib.pyplot as plt
# import deepwave
# from deepwave import scalar

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ny = 2301
nx = 751
dx = 4.0
v = torch.from_file('marmousi_vp.bin',
                    size=ny*nx).reshape(ny, nx).to(device)

下面，我们将指定震源和接收器的位置，和使用的地震子波

Deepwave可以同时模拟所有的震源。

这纯粹是为了提高计算性能——它们之间不相互作用，结果应该是相同的，如果震源是单独传播的。

同时传播的炮假设有相同数量的震源和接收器，因此我们可以在张量中分别提供它们的位置 [shot, source, space], 和 [shot, receiver, space], 并且在张量中提供源振幅 [shot, source, time]. 

许多应用程序每个炮只有一个源，因此源的尺寸将是单位长度, 但是Deepwave提供了更多的选项:

In [ ]:
n_shots = 115

n_sources_per_shot = 1
d_source = 20  # 20 * 4m = 80m
first_source = 10  # 10 * 4m = 40m
source_depth = 2  # 2 * 4m = 8m

n_receivers_per_shot = 384
d_receiver = 6  # 6 * 4m = 24m
first_receiver = 0  # 0 * 4m = 0m
receiver_depth = 2  # 2 * 4m = 8m

freq = 25
nt = 750
dt = 0.004
peak_time = 1.5 / freq

In [ ]:
# 震源位置
source_locations = torch.zeros(n_shots, n_sources_per_shot, 2,
                               dtype=torch.long, device=device)
source_locations[..., 1] = source_depth
source_locations[:, 0, 0] = torch.arange(n_shots) * d_source + first_source

In [ ]:
source_locations

In [ ]:
# 接收器位置
receiver_locations = torch.zeros(n_shots, n_receivers_per_shot, 2,
                                 dtype=torch.long, device=device)
receiver_locations[..., 1] = receiver_depth
receiver_locations[:, :, 0] = (
    (torch.arange(n_receivers_per_shot) * d_receiver + first_receiver)
    .repeat(n_shots, 1)
)

In [ ]:
receiver_locations

In [ ]:
# 震源的振幅
source_amplitudes = (
    deepwave.wavelets.ricker(freq, nt, dt, peak_time)
    .repeat(n_shots, n_sources_per_shot, 1)
    .to(device)
)

In [ ]:
# 正演模拟
out = scalar(v, dx, dt, source_amplitudes=source_amplitudes,
             source_locations=source_locations,
             receiver_locations=receiver_locations,
             accuracy=8,
             pml_freq=freq)

In [ ]:
# 画图
receiver_amplitudes = out[-1]
vmin, vmax = torch.quantile(receiver_amplitudes[0],
                            torch.tensor([0.05, 0.95]).to(device))
_, ax = plt.subplots(1, 2, figsize=(10.5, 7), sharey=True)
ax[0].imshow(receiver_amplitudes[57].cpu().T, aspect='auto', cmap='gray',
             vmin=vmin, vmax=vmax)
ax[1].imshow(receiver_amplitudes[:, 192].cpu().T, aspect='auto', cmap='gray',
             vmin=vmin, vmax=vmax)
ax[0].set_xlabel("Channel")
ax[0].set_ylabel("Time Sample")
ax[1].set_xlabel("Shot")
plt.tight_layout()
plt.savefig('example_forward_model.jpg')

In [ ]:
# 保存
receiver_amplitudes.cpu().numpy().tofile('marmousi_data.bin')